In [1]:
import pickle
import numpy as np
import re
import pandas as pd
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
news_df = pd.read_csv('/content/WELFake_Dataset.csv', quoting=3, on_bad_lines='skip', header=0)

In [3]:
news_df.head()

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,Unnamed: 0,title,text,label
0,LAW ENFORCEMENT ON HIGH ALERT Following Threats Against Cops And Whites On 9-11By #BlackLivesMatter And #FYF911 Terrorists [VIDEO],"""No comment is expected from Barack Obama Members of the #FYF911 or #FukYoFlag and #BlackLivesMatter movements called for the lynching and hanging of white people and cops. They encouraged others on a radio show Tuesday night to turn the tide and kill white people and cops to send a message about the killing of black people in America.One of the F***YoFlag organizers is called Sunshine. She has a radio blog show hosted from Texas called",Sunshine s F***ing Opinion Radio Show. A snapshot of her #FYF911 @LOLatWhiteFear Twitter page at 9:53 p.m. shows that she was urging supporters to Call now!! #fyf911 tonight we continue to dismantle the illusion of white Below is a SNAPSHOT Twitter Radio Call Invite #FYF911The radio show aired at 10:00 p.m. eastern standard time.During the show,callers clearly call for lynching and killing of white people.A 2:39 minute clip from the radio show can be heard here. It was provided to Breitbart Texas by someone who would like to be referred to as Hannibal. He has already received death threats as a result of interrupting #FYF911 conference calls.An unidentified black man said when those mother f**kers are by themselves,that s when when we should start f***ing them up. Like they do us,when a bunch of them ni**ers takin one of us out,that s how we should roll up. He said,Cause we already roll up in gangs anyway. There should be six or seven black mother f**ckers,see that white person,and then lynch their ass. Let s turn the tables. They conspired that if cops started losing people,then there will be a state of emergency. He speculated that one of two things would happen,a big-ass [R s?????] war,or ni**ers,they are going to start backin up. We are already getting killed out here so what the f**k we got to lose? Sunshine could be heard saying,Yep,that s true. That s so f**king true. He said,We need to turn the tables on them. Our kids are getting shot out here. Somebody needs to become a sacrifice on their side.He said,Everybody ain t down for that s**t,or whatever,but like I say,everybody has a different position of war. He continued,Because they don t give a f**k anyway. He said again,We might as well utilized them for that s**t and turn the tables on these n**ers. He said,that way we can start lookin like we ain t havin that many casualties,and there can be more causalities on their side instead of ours. They are out their killing black people,black lives don t matter,that s what those mother f**kers so we got to make it matter to them. Find a mother f**ker that is alone. Snap his ass,and then f***in hang him from a damn tree. Take a picture of it and then send it to the mother f**kers. We just need one example,and then people will start watchin . This will turn the tables on s**t,he said. He said this will start a trickle-down effect. He said that when one white person is hung and then they are just flat-hanging,that will start the trickle-down effect. He continued,Black people are good at starting trends. He said that was how to get the upper-hand. Another black man spoke up saying they needed to kill cops that are killing us. The first black male said,That will be the best method right there. Breitbart Texas previously reported how Sunshine was upset when racist white people infiltrated and disrupted one of her conference calls. She subsequently released the phone number of one of the infiltrators. The veteran immediately started receiving threatening calls.One of the #F***YoFlag movement supporters allegedly told a veteran who infiltrated their publicly posted conference call,We are going to rape and gut your pregnant wife,and your f***ing piece of sh*t unborn creature will be hung from a tree. Breitbart Texas previously encountered Sunshine at a Sandra Bland protest at the Waller County Ja

# About the Dataset:

id: unique id for a news article                                     
title: the title of a news article                                  
author: author of the news article                                    
text: the text of the article; could be incomplete                                        
label: a label that marks whether the news article is real or fake:                              
    1: Fake news                                                 
    0: real News                                        

# 1 Preprocessing

In [4]:
news_df.isnull().sum()

,0
Unnamed: 0,12448
title,12464
text,12479
label,12490


In [5]:
news_df.shape

(12504, 4)

In [6]:
news_df = news_df.fillna(' ')

In [7]:
news_df.isnull().sum()

,0
Unnamed: 0,0
title,0
text,0
label,0


In [8]:
print(news_df.columns)

Index(['Unnamed: 0', 'title', 'text', 'label'], dtype='object')


In [9]:
# Combine 'title' and 'text' columns for richer content (Optional)
news_df['content'] = news_df['title'] + ' ' + news_df['text']



In [10]:
news_df

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,Unnamed: 0,title,text,label,content
0,LAW ENFORCEMENT ON HIGH ALERT Following Threats Against Cops And Whites On 9-11By #BlackLivesMatter And #FYF911 Terrorists [VIDEO],"""No comment is expected from Barack Obama Members of the #FYF911 or #FukYoFlag and #BlackLivesMatter movements called for the lynching and hanging of white people and cops. They encouraged others on a radio show Tuesday night to turn the tide and kill white people and cops to send a message about the killing of black people in America.One of the F***YoFlag organizers is called Sunshine. She has a radio blog show hosted from Texas called",Sunshine s F***ing Opinion Radio Show. A snapshot of her #FYF911 @LOLatWhiteFear Twitter page at 9:53 p.m. shows that she was urging supporters to Call now!! #fyf911 tonight we continue to dismantle the illusion of white Below is a SNAPSHOT Twitter Radio Call Invite #FYF911The radio show aired at 10:00 p.m. eastern standard time.During the show,callers clearly call for lynching and killing of white people.A 2:39 minute clip from the radio show can be heard here. It was provided to Breitbart Texas by someone who would like to be referred to as Hannibal. He has already received death threats as a result of interrupting #FYF911 conference calls.An unidentified black man said when those mother f**kers are by themselves,that s when when we should start f***ing them up. Like they do us,when a bunch of them ni**ers takin one of us out,that s how we should roll up. He said,Cause we already roll up in gangs anyway. There should be six or seven black mother f**ckers,see that white person,and then lynch their ass. Let s turn the tables. They conspired that if cops started losing people,then there will be a state of emergency. He speculated that one of two things would happen,a big-ass [R s?????] war,or ni**ers,they are going to start backin up. We are already getting killed out here so what the f**k we got to lose? Sunshine could be heard saying,Yep,that s true. That s so f**king true. He said,We need to turn the tables on them. Our kids are getting shot out here. Somebody needs to become a sacrifice on their side.He said,Everybody ain t down for that s**t,or whatever,but like I say,everybody has a different position of war. He continued,Because they don t give a f**k anyway. He said again,We might as well utilized them for that s**t and turn the tables on these n**ers. He said,that way we can start lookin like we ain t havin that many casualties,and there can be more causalities on their side instead of ours. They are out their killing black people,black lives don t matter,that s what those mother f**kers so we got to make it matter to them. Find a mother f**ker that is alone. Snap his ass,and then f***in hang him from a damn tree. Take a picture of it and then send it to the mother f**kers. We just need one example,and then people will start watchin . This will turn the tables on s**t,he said. He said this will start a trickle-down effect. He said that when one white person is hung and then they are just flat-hanging,that will start the trickle-down effect. He continued,Black people are good at starting trends. He said that was how to get the upper-hand. Another black man spoke up saying they needed to kill cops that are killing us. The first black male said,That will be the best method right there. Breitbart Texas previously reported how Sunshine was upset when racist white people infiltrated and disrupted one of her conference calls. She subsequently released the phone number of one of the infiltrators. The veteran immediately started receiving threatening calls.One of the #F***YoFlag movement supporters allegedly told a veteran who infiltrated their publicly posted conference call,We are going to rape and gut your pregnant wife,and your f***ing piece of sh*t unborn creature will be hung from a tree. Breitbart Texas previously encountered Sunshine at a Sandra Bland protest at the Waller C

# separating the data & label

In [11]:
X = news_df.drop('label',axis=1)
y = news_df['label']

In [12]:
print(X)

# Stemming:

Stemming is the process of reducing a word to its Root word

example: hung         hanged        hanging ======hang

# Steps:
lower case                 
splitting                             
removing stopwords                              
stemming                                   

In [13]:
ps = PorterStemmer()
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [ps.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [14]:
news_df['content'] = news_df['content'].apply(stemming)

In [15]:
news_df['content']

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,content
0,LAW ENFORCEMENT ON HIGH ALERT Following Threats Against Cops And Whites On 9-11By #BlackLivesMatter And #FYF911 Terrorists [VIDEO],"""No comment is expected from Barack Obama Members of the #FYF911 or #FukYoFlag and #BlackLivesMatter movements called for the lynching and hanging of white people and cops. They encouraged others on a radio show Tuesday night to turn the tide and kill white people and cops to send a message about the killing of black people in America.One of the F***YoFlag organizers is called Sunshine. She has a radio blog show hosted from Texas called",Sunshine s F***ing Opinion Radio Show. A snapshot of her #FYF911 @LOLatWhiteFear Twitter page at 9:53 p.m. shows that she was urging supporters to Call now!! #fyf911 tonight we continue to dismantle the illusion of white Below is a SNAPSHOT Twitter Radio Call Invite #FYF911The radio show aired at 10:00 p.m. eastern standard time.During the show,callers clearly call for lynching and killing of white people.A 2:39 minute clip from the radio show can be heard here. It was provided to Breitbart Texas by someone who would like to be referred to as Hannibal. He has already received death threats as a result of interrupting #FYF911 conference calls.An unidentified black man said when those mother f**kers are by themselves,that s when when we should start f***ing them up. Like they do us,when a bunch of them ni**ers takin one of us out,that s how we should roll up. He said,Cause we already roll up in gangs anyway. There should be six or seven black mother f**ckers,see that white person,and then lynch their ass. Let s turn the tables. They conspired that if cops started losing people,then there will be a state of emergency. He speculated that one of two things would happen,a big-ass [R s?????] war,or ni**ers,they are going to start backin up. We are already getting killed out here so what the f**k we got to lose? Sunshine could be heard saying,Yep,that s true. That s so f**king true. He said,We need to turn the tables on them. Our kids are getting shot out here. Somebody needs to become a sacrifice on their side.He said,Everybody ain t down for that s**t,or whatever,but like I say,everybody has a different position of war. He continued,Because they don t give a f**k anyway. He said again,We might as well utilized them for that s**t and turn the tables on these n**ers. He said,that way we can start lookin like we ain t havin that many casualties,and there can be more causalities on their side instead of ours. They are out their killing black people,black lives don t matter,that s what those mother f**kers so we got to make it matter to them. Find a mother f**ker that is alone. Snap his ass,and then f***in hang him from a damn tree. Take a picture of it and then send it to the mother f**kers. We just need one example,and then people will start watchin . This will turn the tables on s**t,he said. He said this will start a trickle-down effect. He said that when one white person is hung and then they are just flat-hanging,that will start the trickle-down effect. He continued,Black people are good at starting trends. He said that was how to get the upper-hand. Another black man spoke up saying they needed to kill cops that are killing us. The first black male said,That will be the best method right there. Breitbart Texas previously reported how Sunshine was upset when racist white people infiltrated and disrupted one of her conference calls. She subsequently released the phone number of one of the infiltrators. The veteran immediately started receiving threatening calls.One of the #F***YoFlag movement supporters allegedly told a veteran who infiltrated their publicly posted conference call,We are going to rape and gut your pregnant wife,and your f***ing piece of sh*t unborn creature will be hung from a tree. Breitbart Texas previously encountered Sunshine at a Sandra Bland protest at the Waller County Jail in Texas,where sh

In [16]:
# Separate features and target variable
X = news_df['content']  # Keep X as text data for vectorization
y = news_df['label']

In [17]:
# Replace empty strings and other non-numeric values in 'y' with 0
y = pd.to_numeric(y, errors='coerce').fillna(0).astype(int)


In [18]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
# Create and fit TfidfVectorizer on training data
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)  # Transform training data

In [28]:
# Transform testing data using the same vectorizer
X_test_vec = vectorizer.transform(X_test)  # Transform testing data

In [21]:
print(X)

0               LAW ENFORCEMENT ON HIGH ALERT Following Threats Against Cops And Whites On 9-11By #BlackLivesMatter And #FYF911 Terrorists [VIDEO]         "No comment is expected from Barack Obama Members of the #FYF911 or #FukYoFlag and #BlackLivesMatter movements called for the lynching and hanging of white people and cops. They encouraged others on a radio show Tuesday night to  turn the tide  and kill white people and cops to send a message about the killing of black people in America.One of the F***YoFlag organizers is called  Sunshine.  She has a radio blog show hosted from Texas called    Sunshine s F***ing Opinion Radio Show. A snapshot of her #FYF911 @LOLatWhiteFear Twitter page at 9:53 p.m. shows that she was urging supporters to  Call now!! #fyf911 tonight we continue to dismantle the illusion of white Below is a SNAPSHOT Twitter Radio Call Invite   #FYF911The radio show aired at 10:00 p.m. eastern standard time.During the show   callers clearly call for  lynching  and  kill

In [22]:
X_train.shape

(10003,)

In [23]:
# Train the model
model = LogisticRegression()
model.fit(X_train_vec, y_train)  # Use the vectorized training data

LogisticRegression()

In [24]:
# on training set
train_y_pred = model.predict(X_train_vec) # Changed X_train to X_train_vec
print(accuracy_score(train_y_pred,y_train)) # Changed Y_train to y_train

0.9993002099370188


In [25]:
# on testing set
testing_y_pred = model.predict(X_test_vec)  # Use X_test_vec instead of X_test
print(accuracy_score(testing_y_pred,y_test))  # Assuming y_test is the correct variable for test labels

0.9992003198720512


In [29]:
# Save the model using pickle
filename = 'fake_news_model.pkl'  # Changed the extension to .pkl
pickle.dump(model, open(filename, 'wb'))